<center> <h1><font size=7> Case Study E</font> </h1> </center>

# Clustering Facebook Posts - Example Answer

This notebook contains a minimum example response to the questions posed in Case Study D. These are not the only ways to approach the problems, but should hopefully point you in a fruitful direction.

# Exploratory tasks

**1. Load and explore the variables distributions in the data set, convert `"status_published"` to a datetime if it is not already.**


In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
posts = pd.read_csv("../../data/fb_sale_posts.csv")

posts.shape

In [ ]:
posts.describe(include="all")

In [ ]:
posts.head()

In [ ]:
posts.dtypes

Exploring: Status Published

In [ ]:
posts["status_published"] = pd.to_datetime(posts["status_published"])

In [ ]:
posts["status_published"].value_counts().sort_index().cumsum().plot()
plt.title("Cumulative sum of posts");

In [ ]:
posts["status_published"].hist(bins=50)
plt.title("Indicator of post frequency");

Exploring: status id

Unique ID, so distribution isn't interesting.

In [ ]:
print(len(posts))
posts["status_id"].nunique()

Exploring: status type

In [ ]:
posts["status_type"].value_counts().plot(kind="bar")
plt.title("Count of status types");

Exploring: num reactions, num likes, num loves, num wows, num hahas, num sads and num angrys

In [ ]:
posts["num_reactions"].hist(bins=50);

Highly skewed distribution. Potentially a powerlaw or exponential decay function.

In [ ]:
columns = ["num_likes", "num_loves", "num_wows", "num_hahas", "num_sads", "num_angrys"]

print("Cutting is end exclusive - '0 to 1' is 0, '1 to 10' is 1-9")

for column in columns:
    plt.figure()
    # bin the data
    out = pd.cut(posts[column], bins=[0, 1, 10, 100, 1000, 10000], include_lowest=True)
    # count how many are in each bin
    ax = out.value_counts(sort=False).plot.bar(rot=45, color="b", figsize=(6,4))
    # change the label to be easier to understand from interval notation
    ax.set_xticklabels([(str(int(c.left)) +" to " + str(int(c.right))) for c in out.cat.categories])
    # set limit to allow comparison
    ax.set_ylim(0,len(posts))
    plt.title(f"{column} distribution")



Explore: num comments, num shares

In [ ]:
# bin the data
out = pd.cut(posts["num_comments"], bins=[0, 1, 10, 100, 1000, 10000], include_lowest=True)
# count how many are in each bin
ax = out.value_counts(sort=False).plot.bar(rot=0, color="g", figsize=(6,4))
# change the label to be easier to understand from interval notation
ax.set_xticklabels([(str(int(c.left)) +" to " + str(int(c.right))) for c in out.cat.categories])
plt.xticks(rotation=30)
# set limit to allow comparison
ax.set_ylim(0,len(posts))
plt.title("num_comments distribution");

More posts with high numbers of comments than reactions. Fewer posts with no comments compared to reaction breakdowns. Some posts do extremely well but the majority have very little interaction.

In [ ]:
# bin the data
out = pd.cut(posts["num_shares"], bins=[0, 1, 10, 100, 1000, 10000], include_lowest=True)
# count how many are in each bin
ax = out.value_counts(sort=False).plot.bar(rot=0, color="g", figsize=(6,4))
# change the label to be easier to understand from interval notation
ax.set_xticklabels([(str(int(c.left)) +" to " + str(int(c.right))) for c in out.cat.categories])
plt.xticks(rotation=30)
# set limit to allow comparison
ax.set_ylim(0,len(posts))
plt.title("num_comments distribution");

Similar trend to the comments, but not as extreme a tail. Interestingly more posts with 100 to 1000 shares than 10 to 100 and comparable to 1 to 10.

**2. Some of the variables are highly skewed numerical distributions. Find methods in `sklearn.preprocessing` or elsewhere to change the distribution of data from skewed to uniform or normal. Use the methods on `"num_comments"`.**

There are a range of methods in `sklearn` and elsewhere than can help us in these sorts of situations.

Converting to normal distribution:

We can use the PowerTransformer with the `"box-cox"` method. For more information on this transformation see [this tutorial](https://www.itl.nist.gov/div898/handbook/eda/section3/eda336.htm). However, the box-cox transformation is defined only for strictly positive values (excludes negative values and zero), but our data contains many zero values. For this reason we will use the `"yeo-johnson"` method. The transformation itself is given in [the sklearn documentation](https://bookdown.org/max/FES/).

In [ ]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method="yeo-johnson", standardize=False)

In [ ]:
posts["num_comments_yeo"] = pt.fit_transform(posts[["num_comments"]])

In [ ]:
posts["num_comments_yeo"].hist()
plt.title("Yeo-Johnson transformation on 'num_comments'");

The above is *more* normally distributed than the orginal data, but as we can see this has not created a perfectly normal distribution. Lets compare it to the original data below.

In [ ]:
posts["num_comments"].hist()
plt.title("Non-transformed 'num_comments'");

Another common approach to this problem would to be to perform a log transformation to reduce the range of the data.

A log function is not defined non-positive values, so we often use `np.log1p` - the `1p` means "one plus". We add 1 to make the data all positive.

Again, we can see that this does not produce a perfect normal distribution in the case of our data (it may be too skewed) - but the spread of data is now much less skewed, and will be easier for models to learn from.

In [ ]:
# using numpy log1p function to the whole column
posts["num_comments"].apply(np.log1p).hist()
plt.title("log1p transformation of 'num_comments'");

Converting to uniform distribution:

We can use a quantile transformation, which splits our data into... quantiles to map to different distributions. In this case we will aim to make a uniform transformation from `"num_comments"`.

In [ ]:
from sklearn.preprocessing import QuantileTransformer

# use default parameters
qt = QuantileTransformer()

In [ ]:
posts["num_comments_uniform"] = qt.fit_transform(posts[["num_comments"]])

In [ ]:
# explore distribution post-transformation
posts["num_comments_uniform"].hist();

The new data is bounded by `[0,1]`. Due to the extreme number of zeros we haven't split the lowest quantile. The rest of the values are distributed uniformly across the domain.

**3. The "num_reactions" should be a sum of all the other reactions recorded per record. Create new features that are the proportion of each reaction type, within the values of 0-1. For example  $𝑝𝑟𝑜𝑝\_𝑎𝑛𝑔𝑟𝑦𝑠 = \frac{𝑛𝑢𝑚\_𝑎𝑛𝑔𝑟𝑦𝑠}{𝑛𝑢𝑚\_𝑟𝑒𝑎𝑐𝑡𝑖𝑜𝑛𝑠}$.**

In [ ]:
columns

In [ ]:
# create a new column for each reaction type
for column in columns:
    # create new name for column using prop_ 
    new_col_name = "prop_" + column.split("_")[1]
    # The proportion is the count of specific breakdown / all reactions
    posts[new_col_name] = posts[column] / posts["num_reactions"]

In [ ]:
proportions = posts[posts.columns[posts.columns.str.startswith("prop_")]]
proportions

In [ ]:
# mean proportion of each reaction type
proportions.mean()

**4. Explore covariances between variables: `"num_reactions", "num_comments", "num_shares", "num_likes", "num_loves"`**

In [ ]:
covariance_exploration = posts[["num_reactions", "num_comments", "num_shares", "num_likes", "num_loves"]]
covariance_exploration

In [ ]:
covariance_exploration.cov()

In [ ]:
covariance_exploration.corr()

Just a few of the insights we can gain from the above:

* num_reactions and num_likes are highly colinear. This is because num_reactions is the sum of all reaction types, and likes are by far the largest reaction. We should keep this colinearity in mind when modelling.
* The number of loves is correlated with the comments and shares, more so than the number of likes. This implies that the reactions other than like may be used in different situations than likes, rather than as a subset of likes.
* Comments and shares are linearly correlated above 0.60 - which may indicate that they are well linked as types of interations. They are far more linked than with reactions.

**Note** We have only looked at the featues without transformation here, further or different insights would be gathered from using the featues in a different transformation.

<strong>5. Create a new set of features and add them to the `posts` dataframe: 
 * "TimeSinceFirst" - (int) count of days since the first day that appears in the data set
 * "TimeOfDay" - (int) hour the post was created
 * "DayOfWeek" - (str) the day of the week the post was made (Monday, Tuesday, Wednesday...)</strong>

**"TimeSinceFirst"**

In [ ]:
# Find the earliest date in the data set
first_date = posts["status_published"].min()

In [ ]:
posts["TimeSinceFirst"] = (posts["status_published"] - first_date).dt.days

In [ ]:
posts["TimeSinceFirst"].hist(bins=50);

**"TimeOfDay"**

In [ ]:
# Each timestamp has an associated hour
posts["TimeOfDay"] = posts["status_published"].dt.hour

In [ ]:
# plot distribution of hours of the day
posts["TimeOfDay"].hist(bins=24);

The question this raises is that does the number of posts peak at around 2am and 7am and is at a minimum at 4pm - or is there a time difference between where the data was recorded and Thailand? It will make a difference to our interpretation of the resulting models / analysis.

**"DayOfWeek"**

We could either convert to integers or to strings. Integers will enforce an interval between values, which makes sense in some sense. For example Monday to Tuesday is the same distance as Thursday to Friday. However, the distance from Sunday to Monday will be large, which is not true. This is due to the cyclical nature of week days. For this reason strings are taken instead and days of the week are treated as categorical for now. They could be binned to lower numbers of categories if necessary (weekend / not-weekend for example).

In [ ]:
posts["DayOfWeek"] = posts["status_published"].dt.day_name()
posts["DayOfWeek"].value_counts(normalize=True) * 100

Similar numbers of posts across all days of the week.

**6. How are the covariances/correlations with numerical features different between status_type == "video", "photo", particularly the numbers of reactions, comments and shares?**

In [ ]:
video = posts[posts["status_type"] == "video"]
photo = posts[posts["status_type"] == "photo"]

In [ ]:
video[["num_reactions", "num_comments", "num_shares"]].corr()

In [ ]:
photo[["num_reactions", "num_comments", "num_shares"]].corr()

In [ ]:
posts.groupby("status_type").size().reset_index().rename(columns={0: "count"})

The correlations between different types of interaction are quite different between the different types of post interaction. 
Briefly:

* The shares of videos are more correlated than with reactions than for photos
* The shares and comments of videos are highly correlated compared to photos

# Modelling

When exploring the proportions of reaction counts we need to keep in mind some contraints of proportions.

If we define a reaction as $react_i$, then the proportion of $react_i$ is $prop\_react_i = \frac{react_i}{num\_reacts}$.

As $\sum^i{react_i} = num\_reacts$

Therefore $\sum^i{prop\_react_i} = 1$

Each proportion has a maximum value of $1$, and the sum of any $react_i$'s must be less than or equal to 1. 

If we plot one proportion of reactions against another, out data will always appear on the line or below $react_y = -react_x + 1$

As likes are by far the most dominant react, it may be useful to define the compliment proportion to likes, which is equivalent to the sum of non-likes proportions.

In [ ]:
posts["prop_not_likes"] = 1 - posts["prop_likes"]

In [ ]:
# likes and non-likes have a relationship (as we defined it)
plt.scatter(posts["prop_likes"], posts["prop_not_likes"]);

In [ ]:
# As likes lower, the loves appear to increase
plt.scatter(posts["prop_likes"], posts["prop_loves"]);

In [ ]:
# There doesn't appear to be a clear relationship
# The two reacts appear to some extent independent
plt.scatter(posts["prop_angrys"], posts["prop_loves"])
plt.xlim(0,1);

In [ ]:
# There appear to be some relationship between the proportions
# of these two reacts, or at least a cluster of 
# values off the axis
plt.scatter(posts["prop_wows"], posts["prop_loves"]);

In [ ]:
# not a lot of crossovers of sad and happy reacts
plt.scatter(posts["prop_hahas"], posts["prop_sads"]);

In [ ]:
# When loves are non-zero there do appear to be haha's
plt.scatter(posts["prop_loves"], posts["prop_hahas"]);

### Normalize Counts

This is a different method than finding the proportions, but it follows a similar intuition. We will divide by the squared sum of each row, rather than the sum as previously (this was given by the total reactions).

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
# fitting does nothing for normalizing (stateless, unlike other scalers)
normal_reacts = Normalizer().fit_transform(posts[columns])

In [ ]:
# Example column vs column plot
plt.scatter(normal_reacts[:,0], normal_reacts[:,1])
plt.xlabel(columns[0])
plt.ylabel(columns[1]);

In [ ]:
from itertools import combinations
# there are nicer plotting methods (such as in seaborn)
# for plotting all columns against each other,
# but this is how I would do it in python standard library
combinations = list(combinations(enumerate(columns), 2))
combinations

In [ ]:
# loop through each combination and plot 
# the normalised counts
for (idx1, col1), (idx2, col2) in combinations:
    plt.figure(figsize=(5,5))
    # plot the index from the array corresponding the the column desired
    plt.scatter(normal_reacts[:,idx1], normal_reacts[:,idx2])
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.title(f"{col1} and {col2}")
    plt.xlim(0,1)
    plt.ylim(0,1);

## Clustering

### Prep

Lets make some functions that will help us with the repetative tasks of plotting and evaluating clustering.

In [ ]:
from sklearn.decomposition import PCA

def plot_data(X, labels=None, components=(0,1)):
    
    # using basic PCA
    pca = PCA(random_state=1)
        
    X_red = pca.fit_transform(X)
    
    plt.figure(figsize=(6,6))
    # select components of reduced data, using cluster label as colouring
    plt.scatter(X_red[:,components[0]], X_red[:,components[1]], c=labels)
    
    plt.xlabel(f"Component {components[0]}")
    plt.ylabel(f"Component {components[1]}")
    
    # add the explained variance of each component selected
    print("Explained variance ratio in shown components:", 
          pca.explained_variance_ratio_[components[0]] + pca.explained_variance_ratio_[components[1]])

In [ ]:
from sklearn.metrics import silhouette_score

def cluster_evaluate(X, cluster_object, tuning_param):
    # initialize clustering object
    clusterer = cluster_object(**tuning_param) # unpack whatever parameters fiven from a dictionary
    
    # generate predictions
    cluster_labels = clusterer.fit_predict(X)
    
    # evaluate
    score = silhouette_score(X=X, labels=cluster_labels)
    
    # return score and labels for exploration
    return score, cluster_labels  

### KMeans

In [ ]:
from sklearn.cluster import KMeans

k_values = list(range(2, 20))
k_scores = []
best_k = None
best_score_kmeans = 0
best_labels_kmeans = None

for k in k_values:   
    individual_score, found_labels = cluster_evaluate(normal_reacts, KMeans, {"n_clusters": k})
    k_scores.append(individual_score)
    
    if individual_score > best_score_kmeans:
        best_score_kmeans = individual_score
        best_k = k
        best_labels_kmeans = found_labels
    
print(f"Best K: {best_k}, with score: {best_score_kmeans}")
    
plt.plot(k_values, k_scores)
plt.xlabel("K")
plt.ylabel("Silhouette score");

In [ ]:
plot_data(normal_reacts, labels=best_labels_kmeans)

Even though our silhouette score is high, we have been unable to differentaite the three clear groupings that are clear from the first components. However, as we are only showing part of the data it may not actually be 3 clear clusters - although the displayed data is a significant proportion of the data's variance. Let's try a different method.

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

eps_values = np.linspace(0.05, 0.6, 10)
eps_scores = []
best_eps = None
best_score_dbscan = 0
best_labels_dbscan = None

for eps in eps_values:   
    individual_score, found_labels = cluster_evaluate(normal_reacts, DBSCAN, {"eps": eps})
    eps_scores.append(individual_score)
    
    if individual_score > best_score_dbscan:
        best_score_dbscan = individual_score
        best_eps = eps
        best_labels_dbscan = found_labels
    
print(f"Best distance: {best_eps}, with score: {best_score_dbscan}")
    
plt.plot(eps_values, eps_scores)
plt.xlabel("Eps")
plt.ylabel("Silhouette score");

In [ ]:
plot_data(normal_reacts, labels=best_labels_dbscan, components=(0,1))

In [ ]:
np.unique(best_labels_dbscan, return_counts=True)

From this investigation we can see that the DBSCAN algorithm out performs the KMeans clustering. In our first two components we can see that it better groups the majority cluster better, as it is not contrained in the same way as kmeans. At first glance there is a mixed cluster found, however, when exploring in differnet components you can see that the two differnetly label data near each other are actually separated in different components.

The resulting clusters are two clusters with some (5) in my case labelled outliers.

### Agglomerative Method

In [ ]:
# Choosing to stick with ward linkage as default

from sklearn.cluster import AgglomerativeClustering

k_values = list(range(2, 10))
k_scores = []
best_k = None
best_score_agglom = 0
best_labels_agglom = None

for k in k_values:   
    individual_score, found_labels = cluster_evaluate(normal_reacts, AgglomerativeClustering, {"n_clusters": k})
    k_scores.append(individual_score)
    
    if individual_score > best_score_agglom:
        best_score_agglom = individual_score
        best_k = k
        best_labels_agglom = found_labels
    
print(f"Best K: {best_k}, with score: {best_score_agglom}")
    
plt.plot(k_values, k_scores)
plt.xlabel("K")
plt.ylabel("Silhouette score");

In [ ]:
plot_data(normal_reacts, labels=best_labels_agglom, components=(0,1))

This gives us 2 clusters as optimal, rather than the 3 from kmeans, or 2 with outliers from DBSCAN. Looking at the first two components the two outlier clusters are grouped together with the half of the majority class. This could be a result of the linkage method, or an effect not shown in our components. Explore the result in components (0,3), there the clusters look like a much more reasonable choice. Further to this, the resulting clusters will be highly impacted by how we normalised the data. Data on a different scale / transformed in a different way will yield different results. Remember: we are not clustering using PCA, we are just using it to visualise the results. 

So, what does this clustering tell us?

My interpretation would be:

* There is a clear majority cluster as shown in the visualisation, which can transfer to two classes in other dimensions.
* There are some outlier clusters which show unusual / well separated values from the main cluster. They are however, not well separated in all dimensions and therefore are mixed in some cases
* For the first principal component by far the largest contributor is the "num_likes", which makes sense from the exploratory analysis

### Exploring groupings

In [ ]:
# taking the best labelling, from DBSCAN (although similar to agglom)

best_labels_dbscan

In [ ]:
# create new frame to analyse
posts_counts = posts[columns].copy()

In [ ]:
# assign colum to the groupings
posts_counts["cluster"] = best_labels_dbscan

In [ ]:
posts_counts.groupby("cluster").size()

In [ ]:
# The clusterings found have what appears to be zero likes
posts_counts.groupby("cluster")["num_likes"].median()

In [ ]:
# getting a summary feel for the different clusters
posts_counts.groupby("cluster").mean()

From this we can see that of the three groups (non-cluster counted as a group in this case), the groups are:

* non-clustered: posts with no likes, but some other reactions (there are five points remember)
* cluster 0: posts with a mix of likes and other reactions
* cluster 1: posts with no likes or reactions

Considering we have not transformed these features at all other than normalisation, this result is quite good! We can group different types of interaction.